In [1]:
# Necessary Imports
import os
import pathlib
import pyrootutils
import pandas as pd
import numpy as np 
import re
import opendatasets as od 

In [2]:
root = pyrootutils.setup_root(os.getcwd(), dotenv=True, pythonpath=True)
data_path = root/"Data"
report_dir = root/"EDA"

In [3]:
report_dir.mkdir(parents=True, exist_ok=True)

In [4]:
def reduce_memory_usage(df, verbose=True):
    """
    Helper method to reduce the memory requirements of the datafrmae
    :param df:
    :param verbose:
    :return:
    """
    numerics = ["int8", "int16", "int32", "int64", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print("Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
df = pd.read_csv(data_path/"pokemon.csv")
df = reduce_memory_usage(df)

Mem. usage decreased to 0.13 Mb (49.4% reduction)


In [6]:
df.sample(20, axis=1).describe().T.style.bar(subset=["mean"], color="#205ff2").background_gradient(subset=["std"], cmap="Reds").background_gradient(subset=["50%"], cmap="coolwarm")

,count,mean,std,min,25%,50%,75%,max
percentage_male,703.000000,55.155762,20.261595,0.000000,50.000000,50.000000,50.000000,100.000000
speed,801.000000,66.334582,28.907662,5.000000,45.000000,65.000000,85.000000,180.000000
hp,801.000000,68.958801,26.576015,1.000000,50.000000,65.000000,80.000000,255.000000
against_normal,801.000000,0.887016,0.266106,0.000000,1.000000,1.000000,1.000000,1.000000
height_m,781.000000,1.163892,1.080328,0.100000,0.600000,1.000000,1.500000,14.500000
experience_growth,801.000000,1054995.905119,160255.835096,600000.000000,1000000.000000,1000000.000000,1059860.000000,1640000.000000
against_psychic,801.000000,1.005306,0.495184,0.000000,1.000000,1.000000,1.000000,4.000000
against_ground,801.000000,1.098003,0.738818,0.000000,1.000000,1.000000,1.000000,4.000000
against_flying,801.000000,1.192884,0.604487,0.250000,1.000000,1.000000,1.000000,4.000000
against_electric,801.000000,1.073970,0.654961,0.000000,0.500000,1.000000,1.000000,4.000000


In [7]:
df2 =  df.sample(5)[["abilities","type1","type2","weight_kg"]]
df2

,abilities,type1,type2,weight_kg
405,"['Natural Cure', 'Poison Point', 'Leaf Guard']",grass,poison,1.200000
757,"['Corrosion', 'Oblivious']",poison,fire,22.200001
796,['Beast Boost'],steel,flying,999.900024
370,"['Rock Head', 'Sheer Force']",dragon,NaN,42.099998
113,"['Chlorophyll', 'Leaf Guard', 'Regenerator']",grass,NaN,35.000000


In [9]:
from ydata_profiling import ProfileReport
ProfileReport(df,title="Profile Report").to_file(str(report_dir/f"features_eda.html"))

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
vals = df.sample(15)["type1"]
vals

205      normal
567      poison
411         bug
701    electric
183       water
217        fire
471      ground
536       water
334      normal
535       water
40       poison
755       grass
698        rock
742         bug
366       water
Name: type1, dtype: object

In [20]:
codes, uniques = pd.factorize(vals)
print("Encoding : ", codes)
print("Unique Categories : ", uniques)

Encoding :  [0 1 2 3 4 5 6 4 0 4 1 7 8 2 4]
Unique Categories :  Index(['normal', 'poison', 'bug', 'electric', 'water', 'fire', 'ground',
       'grass', 'rock'],
      dtype='object')


In [ ]:
# Convert to numeric
obj_cols = list(df.select_dtypes(['object']).columns)
df[obj_cols] = df[obj_cols].apply(pd.to_numeric, errors='ignore', downcast="float")

# Repeat this once more
obj_cols = list(df.select_dtypes(['object']).columns)

In [ ]:
# Exclude booleans or indicators
bool_cols = df.select_dtypes(include=boolean).columns.tolist()
num_cols = df.select_dtypes(include=np.number).columns.tolist()


# Verify and enforce casting
num_cols = [x for x in num_cols if x not in bool_cols]
df[num_cols] = df[num_cols].astype(np.float32)


# Min-max Scaling
from sklearn.preprocessing import minmax_scale
df[num_cols] = minmax_scale(df[num_cols])

In [ ]:
# Time based features
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)
df["current_date"] = pd.to_datetime('today').normalize()
df["elapsed"] = (df["current_date"] - df["date"]) / pd.Timedelta(days=1)


In [35]:
def add_datepart(df, col, drop=False):
    """
    Extract all pieces of the datetime object into columns
    :param df: DataFrame
    :param col: The column that is the timestamp
    :param drop: Delete the existing timestamp column
    :return:
    """
    fld = df[col]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[col] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', col)
    for n in ('Year', 'Month', 'Week', 'Day',
              'Dayofweek', 'Dayofyear',
              'Is_month_end', 'Is_month_start',
              'Is_quarter_end', 'Is_quarter_start',
              'Is_year_end', 'Is_year_start'):
        df[targ_pre +"_"+ n] = getattr(fld.dt, n.lower())
    #df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop:
        df.drop(col, axis=1, inplace=True)



In [36]:
import warnings
warnings.simplefilter(action="ignore")

In [44]:
df2 = pd.read_csv(data_path/"reddit_vm.csv")[["title","timestamp"]]
x = df2.head(5)
x

,title,timestamp
0,Health Canada approves AstraZeneca COVID-19 va...,2021-02-27 06:33:45
1,COVID-19 in Canada: 'Vaccination passports' a ...,2021-02-26 07:11:07
2,Coronavirus variants could fuel Canada's third...,2021-02-21 07:50:08
3,Canadian government to extend COVID-19 emergen...,2021-02-20 06:35:13
4,Canada: Pfizer is 'extremely committed' to mee...,2021-02-16 11:36:28


In [46]:
add_datepart(x, "timestamp", drop=True)
x

,title,timestamp_Year,timestamp_Month,timestamp_Week,timestamp_Day,timestamp_Dayofweek,timestamp_Dayofyear,timestamp_Is_month_end,timestamp_Is_month_start,timestamp_Is_quarter_end,timestamp_Is_quarter_start,timestamp_Is_year_end,timestamp_Is_year_start
0,Health Canada approves AstraZeneca COVID-19 va...,2021,2,8,27,5,58,False,False,False,False,False,False
1,COVID-19 in Canada: 'Vaccination passports' a ...,2021,2,8,26,4,57,False,False,False,False,False,False
2,Coronavirus variants could fuel Canada's third...,2021,2,7,21,6,52,False,False,False,False,False,False
3,Canadian government to extend COVID-19 emergen...,2021,2,7,20,5,51,False,False,False,False,False,False
4,Canada: Pfizer is 'extremely committed' to mee...,2021,2,7,16,1,47,False,False,False,False,False,False


C:\Users\snkni\AppData\Local\Temp\ipykernel_7684\1292848905.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[targ_pre + n] = getattr(fld.dt, n.lower())
C:\Users\snkni\AppData\Local\Temp\ipykernel_7684\1292848905.py:18: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df[targ_pre + n] = getattr(fld.dt, n.lower())
C:\Users\snkni\AppData\Local\Temp\ipykernel_7684\1292848905.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [ ]:
def arc_to_deg(arc):
    """convert spherical arc length [m] to great circle distance [deg]"""
    return float(arc)/6371/1000 * 180/math.pi

def deg_to_arc(deg):
    """convert great circle distance [deg] to spherical arc length [m]"""
    return float(deg)*6371*1000 * math.pi/180

def latlon_to_xyz(lat,lon):
    """Convert angluar to cartesian coordiantes

    latitude is the 90deg - zenith angle in range [-90;90]
    lonitude is the azimuthal angle in range [-180;180]
    """
    r = 6371 # https://en.wikipedia.org/wiki/Earth_radius
    theta = math.pi/2 - math.radians(lat)
    phi = math.radians(lon)
    x = r * math.sin(theta) * math.cos(phi) # bronstein (3.381a)
    y = r * math.sin(theta) * math.sin(phi)
    z = r * math.cos(theta)
    return [x,y,z]

def xyz_to_latlon (x,y,z):
    """Convert cartesian to angular lat/lon coordiantes"""
    r = math.sqrt(x**2 + y**2 + z**2)
    theta = math.asin(z/r) # https://stackoverflow.com/a/1185413/4933053
    phi = math.atan2(y,x)
    lat = math.degrees(theta)
    lon = math.degrees(phi)
    return [lat,lon]

In [ ]:
def geodist(coord1,coord2):
    """
    Calculate the distance between
    (lat1, lon1), (lat2, lon2)
    """
    # Convert to miles 1km = 0.621371 miles
    return round(mpu.haversine_distance(coord1,coord2)*0.621371,2)

In [ ]:
%%time

import numba as nb
@nb.njit(parallel=True,fastmath=True)
def func(len_matrix):
    """
    Create a NxN matrix for distance lookup
    :param len_matrix: Number of data points you have
    :return: 
    """
    zip_dist = np.zeros((len_matrix, ) * 2)
    indices = np.arange(len_matrix)
    # Get upper triangular pairs 
    fill_cells = np.stack(np.triu_indices(len_matrix), axis=1)
    # Loop through upper triangular indices while avoiding diagonal element indices
    for idx in fill_cells:
        i,j = indices[idx]
        if i!=j:
            zip_dist[i][j]= zip_dist[j][i] = geodist(coordinates[i],coordinates[j])
    return zip_dist

In [ ]:
df = pd.read_parquet(featpath, engine='pyarrow')

In [55]:
arr = np.random.random((10000000,40))
df= pd.DataFrame(arr)
arr.shape

(10000000, 40)

### File Save

In [56]:
%%time
# Saving it to csv - 10 million rows
df.to_csv(data_path/"random_data.csv")

CPU times: total: 4min 19s
Wall time: 5min 29s


In [51]:
%%time
# Saving it as numpy array
np.save(data_path/"random_data.npy",arr)

CPU times: total: 484 ms
Wall time: 2.37 s


### File Load

In [59]:
%%time
# Reading 10 million rows from csv
df=pd.read_csv(data_path/"random_data.csv")

CPU times: total: 46.8 s
Wall time: 1min 57s


In [57]:
%%time
# Loading the numpy array
arr = np.load(data_path/"random_data.npy")

CPU times: total: 1.33 s
Wall time: 3.03 s


In [7]:
! pip install pyarrow

  Using cached pyarrow-13.0.0-cp39-cp39-win_amd64.whl (24.4 MB)


In [22]:
num_files = 10
# Run this in a for loop
for idx in range(num_files):
    df = pd.DataFrame(np.random.random((1000000,5)),columns=["A","B","C","D","E"])
    df.to_parquet(data_path/f"data_{str(idx)}.parquet",engine="pyarrow")

file_list = [str(x) for x in sorted(data_path.glob("*.parquet"))]
file_list

['E:\\GIT_ROOT\\Learning\\exploring-pandas-and-numpy\\Data\\data_0.parquet',
 'E:\\GIT_ROOT\\Learning\\exploring-pandas-and-numpy\\Data\\data_1.parquet',
 'E:\\GIT_ROOT\\Learning\\exploring-pandas-and-numpy\\Data\\data_2.parquet',
 'E:\\GIT_ROOT\\Learning\\exploring-pandas-and-numpy\\Data\\data_3.parquet',
 'E:\\GIT_ROOT\\Learning\\exploring-pandas-and-numpy\\Data\\data_4.parquet',
 'E:\\GIT_ROOT\\Learning\\exploring-pandas-and-numpy\\Data\\data_5.parquet',
 'E:\\GIT_ROOT\\Learning\\exploring-pandas-and-numpy\\Data\\data_6.parquet',
 'E:\\GIT_ROOT\\Learning\\exploring-pandas-and-numpy\\Data\\data_7.parquet',
 'E:\\GIT_ROOT\\Learning\\exploring-pandas-and-numpy\\Data\\data_8.parquet',
 'E:\\GIT_ROOT\\Learning\\exploring-pandas-and-numpy\\Data\\data_9.parquet']

### Loading millions of rows in Pandas

In [23]:
%%time 
# Loading chunks into Pandas as a dataframe
df_list = []
for file in file_list:
    df_list.append(pd.read_parquet(file, engine='pyarrow'))
df = pd.concat(df_list)
df.shape

CPU times: total: 219 ms
Wall time: 436 ms


(10000000, 5)

### Loading in Vaex

In [24]:
%%time
# Loading data in Vaex
import vaex
df = vaex.open_many(file_list)
df.shape

CPU times: total: 15.6 ms
Wall time: 38 ms


(10000000, 5)